In [ ]:
import requests
import pandas as pd
from lxml import etree
import time
import os # Neu: Wird für Ordner- und Dateipfade benötigt





overall_start_date = '2022-06-27'
overall_end_date = '2022-06-30'  # Beispielzeitraum: 16. Dezember 2022 bis 17. Dezember 2022


all_daily_dfs = []
base_url = 'https://web-api.tp.entsoe.eu/api'

API_TOKEN = 'f5b63dee-6535-43a6-ae33-42c85ca9e3c6' 
base_params = {
    'securityToken': API_TOKEN,
    'documentType': 'A84',
    'processType': 'A67',
    'businessType': 'A96',
    'Standard_MarketProduct': 'A01',
    'controlArea_Domain': '10YDE-VE-------2',
}

output_folder = 'xml_srl_work_prices'
os.makedirs(output_folder, exist_ok=True)


date_range = pd.date_range(start=overall_start_date, end=overall_end_date, freq='D')
for start_of_day in date_range:
    period_start = start_of_day.strftime('%Y%m%d%H%M')
    period_end = (start_of_day + pd.Timedelta(days=1)).strftime('%Y%m%d%H%M')
    
    current_params = base_params.copy()
    current_params['periodStart'] = period_start
    current_params['periodEnd'] = period_end
    
    print(f"\nLade Daten für den {start_of_day.date()}...")
    
    try:
        response = requests.get(base_url, params=current_params)
        response.raise_for_status()
        xml_text = response.text

        file_name = f"{period_start}-{period_end}.xml"
        filepath = os.path.join(output_folder, f"{file_name}")
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(xml_text)
        print(f"-> XML-Antwort gespeichert in '{filepath}'")

    except requests.exceptions.HTTPError as e:
        print(f"-> FEHLER beim Download für {start_of_day.date()}: {e}")
    except Exception as e:
        print(f"-> Ein unerwarteter FEHLER ist aufgetreten: {e}")
        
    print("Warte 1 Sekunde...")
    time.sleep(0.5)

if all_daily_dfs:
    print("\nAlle Downloads abgeschlossen. Füge tägliche Daten zusammen...")
    final_df = pd.concat(all_daily_dfs)
    print("\nDownload und Verarbeitung abgeschlossen. Finales DataFrame:")
    print(final_df.info())
    print(final_df.head())
else:
    print("\nDownload-Prozess beendet, aber es wurden keine Daten gefunden.")